# Sesión 3.3 - Scrapy - Web noticias UM
En este ejemplo vamos a definir un Spider sencillo para obtener las noticias de la Universidad de Murcia que se encuentran en la URL https://www.um.es/web/sala-prensa/notas-de-prensa/historico?type=com.liferay.journal.model.JournalArticle&category=4753007

Para eso deberemos ver el código de la página y su estructura para encontrar la información que queremos extraer.

Una vez sabemos eso, podemos definir nuestra clase UMSpider.


In [ ]:
!pip3 install -U scrapy

In [1]:
import scrapy
import sys
import json
import locale
import time
import random
from bs4 import BeautifulSoup

# Para cada crawler que nos definimos nos debemos crear una clase Spider que debe heredar de la clase scrapy.Spider

class UMSpider (scrapy.Spider):
    name = 'unimurcia'

    # Decimos que el dominio válido es el de la UM
    allowed_domains = ['www.um.es']

    # podemos definir las páginas de inicio
    start_urls = ['https://www.um.es/web/sala-prensa/notas-de-prensa/historico?type=com.liferay.journal.model.JournalArticle&category=4753007']

    # para evitar que el sitio te bloquee por usar scrapy es interesante cambiar el USER_AGENT
    # El user agent por defecto de Scrapy cuando hace una petición es
    # Scrapy/VERSION (+https://scrapy.org)
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }

    # debemos de implementar este método que se llamará para cada una de las páginas que se vayan analizando
    def parse (self, response):
        """
        @inherit

        @param self
        @param response
        """

        # Guardamos la URL del sitio que se está visitando
        url = str(response.request.url).strip()

        # Cogemos el contenido relevante y para eso debemos usar selectores CSS
        for article in response.css ('XXXXXXXXXXX'):
            # Cogemos el contenido del título
            title = str (article.css ('XXXXXXXXXXXX').get ()).strip()
            title = BeautifulSoup (title, 'html.parser').get_text().strip()
            # Cogemos el contenido del contenido
            content = "".join (article.css ('XXXXXXXXX').get ())
            content = BeautifulSoup (content, 'html.parser').get_text().strip().replace("\"","")

            # Cogemos también la fecha si existe
            date = str(article.css ('XXXXXXXXXXXX').get().strip())
            if (date):
                date = BeautifulSoup (date, 'html.parser').get_text().strip()

            print ("-------------------------")
            print (url)
            print (title)
            print (content)
            print (date)
            print ("-------------------------")

            data = {
                'url' : url,
                'title': title,
                'content': content,
                'date': date
            }

            filename = str(random.random()).replace(".","") + ".json"

            # Guardamos el documento si tiene contenido y título
            if content and title:
                with open ('umu/' + filename, 'w') as f:
                    json.dump (data, f, indent = 4)


        # Obtenemos todas las otros links de la página representados por la etiqueta <a>
        url_in_current_document = response.css ('a')
        for next_page in url_in_current_document:
            # Para limitar que solamente se parseen las noticias dentro de 'https://www.um.es/web/sala-prensa/'
            # obtenemos el atributo href de la etiqueta <a> y parseamos la página
            yield response.follow (next_page, self.parse)


## Apartado 1.2
Para poder lanzar el Spider necesitamos que ejecutar el siguiente código donde se configuará y lanzará el proceso.
Hay que hacer notar que solamente se puede lanzar un proceso por cada sesión en Jupyter notebook es por eso por lo que se recomienda exportar el código en un script de Python .py para poder ejecutarlo desde la línea de comandos.

In [2]:
import scrapy, os
from scrapy.crawler import CrawlerProcess

# Creamos un proceso de Crawler podemos poner distintas settings que están definidas en la documentación.
# Entre ellas podemos ocular los logs del proceso de Crawling.
process = CrawlerProcess(settings={
    "LOG_ENABLED": False,
    # Used for pipeline 1
})

# Como se ha definido anteriormente en el RSSCrawler, los ficheros se van a almacenar en la carpeta "rss"
# Comprobamos que existe la carpeta y si no existe la creamos
if (os.path.exists('umu')== False):
    os.mkdir('umu')

# Creamos el proceso con el RSSSpider
process.crawl(UMSpider)
# Ejecutamos el Crawler
process.start()

INFO:scrapy.utils.log:Scrapy 2.11.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0], pyOpenSSL 23.2.0 (OpenSSL 3.1.3 19 Sep 2023), cryptography 41.0.4, Platform Linux-5.15.120+-x86_64-with-glibc2.35
INFO:scrapy.addons:Enabled addons:
[]
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)
DEBUG:scrap

-------------------------
https://www.um.es/web/sala-prensa/-/la-facultad-de-ciencias-del-trabajo-de-la-umu-celebra-el-x-foro-de-empleo/1.1?redirect=%2Fweb%2Fsala-prensa%2Fnotas-de-prensa%2Fhistorico%3Ftype%3Dcom.liferay.journal.model.JournalArticle%26category%3D4753007
La Facultad de Ciencias del Trabajo de la UMU celebra el X Foro de Empleo
La Facultad de Ciencias del Trabajo de la Universidad de Murcia ha celebrado este jueves, 26 de octubre, el X Foro Regional de Empleo y Desarrollo Local de la Región de Murcia, organizado en colaboración con la Cátedra de Agenda Urbana y Desarrollo Estratégico Local, el Observatorio de Desarrollo Rural, Local y Empleo y la Asociación de Agentes de Empleo y Desarrollo Local de la Región de Murcia. Este Foro de Empleo se ha consolidado ya como un punto de encuentro y debate entre estudiantes, profesores y técnicos de empleo y desarrollo local que trabajan o tienen interés en la promoción económica y la creación de empleo.
En esta ocasión, el Foro de

DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.um.es/web/sala-prensa/-/el-director-de-cine-graduado-en-la-umu-%C3%81ngel-morales-gana-el-premio-tve-la-aventura-del-saber-/1.2?redirect=%2Fweb%2Fsala-prensa%2Fnotas-de-prensa%2Fhistorico%3Ftype%3Dcom.liferay.journal.model.JournalArticle%26category%3D4753007> (referer: https://www.um.es/web/sala-prensa/notas-de-prensa/historico?type=com.liferay.journal.model.JournalArticle&category=4753007)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.um.es/web/sala-prensa/actualidad> (referer: https://www.um.es/web/sala-prensa/notas-de-prensa/historico?type=com.liferay.journal.model.JournalArticle&category=4753007)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.um.es/web/sala-prensa/-/el-rector-recibe-a-seis-catedr%C3%A1ticos-de-la-universidad-china-de-hangzhou/1.1?redirect=%2Fweb%2Fsala-prensa%2Fnotas-de-prensa%2Fhistorico%3Ftype%3Dcom.liferay.journal.model.JournalArticle%26category%3D4753007> (referer: https://www.um.es/web/

-------------------------
https://www.um.es/web/sala-prensa/-/la-sede-electr%C3%B3nica-de-la-universidad-de-murcia-finalista-en-los-premios-cnis-2023/1.0?redirect=%2Fweb%2Fsala-prensa%2Fnotas-de-prensa%2Fhistorico%3Ftype%3Dcom.liferay.journal.model.JournalArticle%26category%3D4753007
La sede electrónica de la Universidad de Murcia, finalista en los premios CNIS 2023
La Sede Electrónica de la Universidad de Murcia ha quedado finalista en los prestigiosos Premios CNIS 2023, entre más de un centenar de candidaturas, que reconocen la excelencia en la innovación digital.
La Universidad de Murcia lleva trabajando en el ámbito de la Administración Digital desde el año 2005, con la publicación del 'Plan Hacia la Administración Electrónica'. De este han emanado un conjunto de sistemas comunes y servicios electrónicos, que permiten ofrecer un volumen significativo de trámites electrónicos para optimizar la gestión administrativa, dirigidos tanto a estudiantes, como al personal propio y organismo

DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.um.es/web/sala-prensa/-/la-facultad-de-ciencias-del-trabajo-de-la-umu-celebra-el-x-foro-de-empleo> (referer: https://www.um.es/web/sala-prensa/-/la-facultad-de-ciencias-del-trabajo-de-la-umu-celebra-el-x-foro-de-empleo/1.1?redirect=%2Fweb%2Fsala-prensa%2Fnotas-de-prensa%2Fhistorico%3Ftype%3Dcom.liferay.journal.model.JournalArticle%26category%3D4753007)


...

DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.um.es/web/sala-prensa/notas-de-prensa/historico?p_p_id=com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_3YlyhdueBzXi&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_3YlyhdueBzXi_delta=20&p_r_p_resetCur=false&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_3YlyhdueBzXi_cur=21> (referer: https://www.um.es/web/sala-prensa/notas-de-prensa/historico?type=com.liferay.journal.model.JournalArticle&category=4753007)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.um.es/web/sala-prensa/notas-de-prensa/historico?p_p_id=com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_3YlyhdueBzXi&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_3YlyhdueBzXi_delta=20&p_r_p_resetCur=false&_com_liferay_asset_publisher_web_portlet_AssetPublisherPort

-------------------------
https://www.um.es/web/sala-prensa/-/la-sede-electr%C3%B3nica-de-la-universidad-de-murcia-finalista-en-los-premios-cnis-2023
La sede electrónica de la Universidad de Murcia, finalista en los premios CNIS 2023
La Sede Electrónica de la Universidad de Murcia ha quedado finalista en los prestigiosos Premios CNIS 2023, entre más de un centenar de candidaturas, que reconocen la excelencia en la innovación digital.
La Universidad de Murcia lleva trabajando en el ámbito de la Administración Digital desde el año 2005, con la publicación del 'Plan Hacia la Administración Electrónica'. De este han emanado un conjunto de sistemas comunes y servicios electrónicos, que permiten ofrecer un volumen significativo de trámites electrónicos para optimizar la gestión administrativa, dirigidos tanto a estudiantes, como al personal propio y organismos con los que se relaciona. De esta manera, se facilita la relación electrónica de los ciudadanos y organismos por medios electrónicos,

DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.um.es/web/sala-prensa/-/el-ranking-de-la-universidad-de-stanford-sit%C3%BAa-a-62-investigadores-de-la-universidad-de-murcia-en-la-%C3%A9lite-mundial-por-el-impacto-de-sus-trabajos/1.7?redirect=%2Fweb%2Fsala-prensa%2Fnotas-de-prensa%2Fhistorico> (referer: https://www.um.es/web/sala-prensa/notas-de-prensa/historico)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.um.es/web/sala-prensa/-/vuelve-la-segunda-edici%C3%B3n-del-festival-umurgentes-como-trampol%C3%ADn-para-el-talento-joven-murciano/1.0?redirect=%2Fweb%2Fsala-prensa%2Fnotas-de-prensa%2Fhistorico> (referer: https://www.um.es/web/sala-prensa/notas-de-prensa/historico)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.um.es/web/sala-prensa/notas-de-prensa/historico?p_p_id=com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_3YlyhdueBzXi&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_I

-------------------------
https://www.um.es/web/sala-prensa/-/-gastrociencia-vuelve-a-la-sala-de-catas-de-estrella-levante-para-explorar-los-secretos-de-los-salazones-mediterr%C3%A1neas/1.3?redirect=%2Fweb%2Fsala-prensa%2Fnotas-de-prensa%2Fhistorico
‘GastroCiencia’ vuelve a la Sala de Catas de Estrella Levante para explorar los secretos de los salazones mediterráneas
El catedrático y divulgador José Manuel López Nicolás, acompañado por Ana Garre, CEO de la multipremiada empresa Salazones Garre de San Pedro del Pinatar, presentan el tercer evento GastroCiencia en la Sala de Catas de Estrella Levante. El bonito, la mojama o la bottarga son productos muy murcianos que se han consumido durante siglos en la región. Por eso, diferentes disciplinas científicas como la tecnología alimentaria se encargan de estudiar cómo la sal y otros ingredientes pueden ayudar a conservar estos alimentos tan ligados a la gastronomía de esta tierra.
El vicerrector de Transferencia, Comunicación y Divulgación C

DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.um.es/web/sala-prensa/notas-de-prensa/historico?p_p_id=com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_3YlyhdueBzXi&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_3YlyhdueBzXi_delta=20&p_r_p_resetCur=false&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_3YlyhdueBzXi_cur=18> (referer: https://www.um.es/web/sala-prensa/notas-de-prensa/historico?type=com.liferay.journal.model.JournalArticle&category=4753007)
DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.um.es/web/sala-prensa/galeria?p_p_id=com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_I8lLFER6Gea8&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_I8lLFER6Gea8_delta=20&p_r_p_resetCur=false&_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_I8lLF